In [3]:
%run Package.ipynb

In [ ]:
def mutual_info(X,Y,select_k):
    
    X = pd.DataFrame(X)
    if select_k >= 1:
        sel_ = SelectKBest(mutual_info_classif, k=select_k).fit(X,Y)
        col = X.columns[sel_.get_support()]
        
    elif 0 < select_k < 1:
        sel_ = SelectPercentile(mutual_info_classif, percentile=select_k*100).fit(X,Y)
        col = X.columns[sel_.get_support()]
        
    else:
        raise ValueError("select_k must be a positive number")
    
    print(col)
    return col

In [ ]:
def RFS(X,Y,estimator,select_k,step):
    X = pd.DataFrame(X)
    estimator = estimator
    rfe = RFE(estimator=estimator, n_features_to_select=select_k, step=step)
    rfe.fit(X, Y)
    selected_feat = X.columns[(rfe.get_support())]
    print(selected_feat)
    return selected_feat

In [ ]:
from sklearn import svm

def RFE_SVM_AUC(X_train,y_train,X_test,y_test,tol):
    
    X_train = pd.DataFrame(X_train)
    X_test = pd.DataFrame(X_test)
    
    features_to_remove = []
    count = 1
    # initial model using all the features
    model_all_features = svm.SVC(kernel='linear',probability=True)
    model_all_features.fit(X_train, y_train) 
    auc_score_all = model_all_features.score(X_test, y_test) # svm_auc_score(y_true, y_scores)
    
    for feature in X_train.columns:
        print()
        print('testing feature: ', feature, ' which is feature ', count,
          ' out of ', len(X_train.columns))
        count += 1
        model = svm.SVC(kernel='linear',probability=True)
        
        model.fit(X_train.drop(features_to_remove + [feature], axis=1), y_train) 
        auc_score_int = model.score(X_test.drop(features_to_remove + [feature], axis=1), y_test)
        print('New Test SVM AUC={}'.format((auc_score_int)))
    
        # print the original svm-auc with all the features
        print('All features Test SVM AUC={}'.format((auc_score_all)))
    
        # determine the drop in the svm-auc
        diff_auc = auc_score_all - auc_score_int
    
        # compare the drop in svm-auc with the tolerance
        if diff_auc >= tol:
            print('Drop in SVM AUC={}'.format(diff_auc))
            print('keep: ', feature)
            
        else:
            print('Drop in SVM AUC={}'.format(diff_auc))
            print('remove: ', feature)
            
            # if the drop in the svm is small and we remove the
            # feature, we need to set the new svm to the one based on
            # the remaining features
            auc_score_all = auc_score_int
            
            # and append the feature to remove to the list
            features_to_remove.append(feature)
    print('DONE!!')
    print('total features to remove: ', len(features_to_remove))  
    features_to_keep = [x for x in X_train.columns if x not in features_to_remove]
    print('total features to keep: ', len(features_to_keep))
    
    return features_to_keep
